In [23]:
import numpy as np
import pandas as pd
import timeit

### original text

In [24]:
def calculate_frequenties(train_orig,test_orig):
    tic0=timeit.default_timer()
    df1 = train_orig[['question1']].copy()
    df2 = train_orig[['question2']].copy()
    df1_test = test_orig[['question1']].copy()
    df2_test = test_orig[['question2']].copy()

    df2.rename(columns = {'question2':'question1'},inplace=True)
    df2_test.rename(columns = {'question2':'question1'},inplace=True)

    train_questions = df1.append(df2)
    train_questions = train_questions.append(df1_test)
    train_questions = train_questions.append(df2_test)
    #train_questions.drop_duplicates(subset = ['qid1'],inplace=True)
    train_questions.drop_duplicates(subset = ['question1'],inplace=True)

    train_questions.reset_index(inplace=True,drop=True)
    questions_dict = pd.Series(train_questions.index.values,index=train_questions.question1.values).to_dict()
    train_cp = train_orig.copy()
    test_cp = test_orig.copy()
    train_cp.drop(['qid1','qid2'],axis=1,inplace=True)

    test_cp['is_duplicate'] = -1
    test_cp.rename(columns={'test_id':'id'},inplace=True)
    comb = pd.concat([train_cp,test_cp])

    comb['q1_hash'] = comb['question1'].map(questions_dict)
    comb['q2_hash'] = comb['question2'].map(questions_dict)

    q1_vc = comb.q1_hash.value_counts().to_dict()
    q2_vc = comb.q2_hash.value_counts().to_dict()

    def try_apply_dict(x,dict_to_apply):
        try:
            return dict_to_apply[x]
        except KeyError:
            return 0
    #map to frequency space
    comb['q1_freq'] = comb['q1_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))
    comb['q2_freq'] = comb['q2_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))

    train_comb = comb[comb['is_duplicate'] >= 0][['id','q1_hash','q2_hash','q1_freq','q2_freq','is_duplicate']]
    test_comb = comb[comb['is_duplicate'] < 0][['id','q1_hash','q2_hash','q1_freq','q2_freq']]

    corr_mat = train_comb.corr()
    return train_comb, test_comb, corr_mat

In [25]:
train_orig =  pd.read_csv('data/train.csv', header=0)
test_orig =  pd.read_csv('data/test.csv', header=0)


In [26]:
train_comb, test_comb, corr_mat = calculate_frequenties(train_orig,test_orig)

In [27]:
corr_mat

,id,q1_hash,q2_hash,q1_freq,q2_freq,is_duplicate
id,1.000000,0.692730,0.286969,-0.001608,-0.000777,-0.008784
q1_hash,0.692730,1.000000,0.492993,-0.341777,-0.202545,-0.206498
q2_hash,0.286969,0.492993,1.000000,-0.392605,-0.466434,-0.349626
q1_freq,-0.001608,-0.341777,-0.392605,1.000000,0.494315,0.296621
q2_freq,-0.000777,-0.202545,-0.466434,0.494315,1.000000,0.198609
is_duplicate,-0.008784,-0.206498,-0.349626,0.296621,0.198609,1.000000


In [31]:
train_comb.to_csv("data/question_frequency_train.csv",index=False)

In [32]:
test_comb.to_csv("data/question_frequency_test.csv",index=False)

### Cleaned vA

In [18]:
train_clean_A =  pd.read_csv('data/train_clean_vA.csv', header=0)
test_clean_A =  pd.read_csv('data/test_clean_vA.csv', header=0)


In [19]:
train_clean_A.rename(columns={'q1_clean_vA':'question1', 'q2_clean_vA':'question2'},inplace=True)
test_clean_A.rename(columns={'q1_clean_vA':'question1', 'q2_clean_vA':'question2'},inplace=True)


In [21]:
len(train_clean_A),len(test_clean_A)

(404290, 2345796)

In [23]:
_, _ , corr_mat2 = calculate_frequenties(train_clean_A,test_clean_A)

In [24]:
corr_mat2

,id,q1_hash,q2_hash,q1_freq,q2_freq,is_duplicate
id,1.000000,0.690644,0.284510,-0.001176,-0.001258,-0.008784
q1_hash,0.690644,1.000000,0.491655,-0.213964,-0.147792,-0.206274
q2_hash,0.284510,0.491655,1.000000,-0.242893,-0.340867,-0.347962
q1_freq,-0.001176,-0.213964,-0.242893,1.000000,0.302562,0.175250
q2_freq,-0.001258,-0.147792,-0.340867,0.302562,1.000000,0.137030
is_duplicate,-0.008784,-0.206274,-0.347962,0.175250,0.137030,1.000000


### Cleaned vB

In [25]:
train_clean_B=  pd.read_csv('data/train_clean_vB.csv', header=0)
test_clean_B =  pd.read_csv('data/test_clean_vB.csv', header=0)


In [28]:
train_clean_B.rename(columns={'q1_clean_vB':'question1', 'q2_clean_vB':'question2'},inplace=True)
test_clean_B.rename(columns={'q1_clean_vB':'question1', 'q2_clean_vB':'question2'},inplace=True)


In [30]:
_, _ , corr_mat3 = calculate_frequenties(train_clean_B,test_clean_B)
corr_mat3

,id,q1_hash,q2_hash,q1_freq,q2_freq,is_duplicate
id,1.000000,0.692021,0.285729,-0.001367,-0.000256,-0.008784
q1_hash,0.692021,1.000000,0.491649,-0.297582,-0.197499,-0.207086
q2_hash,0.285729,0.491649,1.000000,-0.340953,-0.456914,-0.353699
q1_freq,-0.001367,-0.297582,-0.340953,1.000000,0.422082,0.257274
q2_freq,-0.000256,-0.197499,-0.456914,0.422082,1.000000,0.193832
is_duplicate,-0.008784,-0.207086,-0.353699,0.257274,0.193832,1.000000


It seems the more it's cleaned the less the correlation; probably my cleaning function makes different questions look the same, hence reducing the score. 

## keyword extraction 

In [31]:
from gensim.summarization import summarize, keywords

In [40]:
qid = 100 

train_orig.ix[qid].question1, train_orig.ix[qid].question2

('Will there really be any war between India and Pakistan over the Uri attack? What will be its effects?',
 'Will there be a nuclear war between India and Pakistan?')